In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3tand1k/DATASET/1.2.2026.listens
/kaggle/input/3tand1k/DATASET/8.1.2026.listens
/kaggle/input/3tand1k/DATASET/17.1.2026.listens
/kaggle/input/3tand1k/DATASET/10.1.2026.listens
/kaggle/input/3tand1k/DATASET/24.1.2026.listens
/kaggle/input/3tand1k/DATASET/29.1.2026.listens
/kaggle/input/3tand1k/DATASET/13.1.2026.listens
/kaggle/input/3tand1k/DATASET/19.1.2026.listens
/kaggle/input/3tand1k/DATASET/9.1.2026.listens
/kaggle/input/3tand1k/DATASET/2.2.2026.listens
/kaggle/input/3tand1k/DATASET/30.1.2026.listens
/kaggle/input/3tand1k/DATASET/27.1.2026.listens
/kaggle/input/3tand1k/DATASET/15.1.2026.listens
/kaggle/input/3tand1k/DATASET/12.1.2026.listens
/kaggle/input/3tand1k/DATASET/18.1.2026.listens
/kaggle/input/3tand1k/DATASET/4.2.2026.listens
/kaggle/input/3tand1k/DATASET/5.2.2026.listens
/kaggle/input/3tand1k/DATASET/23.1.2026.listens
/kaggle/input/3tand1k/DATASET/11.1.2026.listens
/kaggle/input/3tand1k/DATASET/readdata.py
/kaggle/input/3tand1k/DATASET/20.1.2026.listens
/kag

In [2]:
# Cài đặt pyspark (thường Kaggle đã có, nhưng chạy để chắc chắn)
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, count, when, lit

# Khởi tạo Spark Session (Cấu hình để tận dụng tối đa RAM của Kaggle)
spark = SparkSession.builder \
    .appName("ChurnPrediction_MassiveData") \
    .config("spark.driver.memory", "16g") \
    .getOrCreate()

print("Spark Session Created Successfully!")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/09 08:05:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session Created Successfully!


In [3]:
# 1. Cấu hình Spark để phân biệt 'trackNumber' và 'TrackNumber'
spark.conf.set("spark.sql.caseSensitive", "true")

# 2. Đọc lại file
# Lưu ý: Thêm option mode="PERMISSIVE" để nếu dòng nào lỗi quá nặng thì Spark sẽ bỏ qua thay vì dừng chương trình
df = spark.read.option("mode", "PERMISSIVE").json("/kaggle/input/3tand1k/DATASET/22.1.2026.listens")

# 3. Kiểm tra xem đã đọc được chưa
df.printSchema()

root
 |-- recording_msid: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- track_metadata: struct (nullable = true)
 |    |-- additional_info: struct (nullable = true)
 |    |    |-- albumartist: string (nullable = true)
 |    |    |-- artist_mbid: string (nullable = true)
 |    |    |-- artist_mbids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- artist_msid: string (nullable = true)
 |    |    |-- artist_names: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- artists_mbids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- choosen_by_user: long (nullable = true)
 |    |    |-- comment: string (nullable = true)
 |    |    |-- date: string (nullable = true)
 |    |    |-- discnumber: string (nullable = true)
 |    |    |-- duration: long (nullable = true)
 |    |    |-- duration_ms: long (nullable = true)
 |    |    |-- dur

26/02/09 08:06:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
# Đường dẫn dữ liệu trên Kaggle thường nằm trong thư mục /kaggle/input/
# Bạn cần check đường dẫn chính xác ở thanh bên phải (Data panel)
json_file_path = "/kaggle/input/3tand1k/DATASET/22.1.2026.listens" 

# Đọc JSON (Spark tự động suy luận schema - cấu trúc)
# multiline=True nếu 1 object json nằm trên nhiều dòng
# multiline=False nếu mỗi dòng là 1 object json (như mẫu bạn gửi)
df = spark.read.json(json_file_path, multiLine=False)

# In cấu trúc dữ liệu ra xem
df.printSchema()

root
 |-- recording_msid: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- track_metadata: struct (nullable = true)
 |    |-- additional_info: struct (nullable = true)
 |    |    |-- albumartist: string (nullable = true)
 |    |    |-- artist_mbid: string (nullable = true)
 |    |    |-- artist_mbids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- artist_msid: string (nullable = true)
 |    |    |-- artist_names: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- artists_mbids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- choosen_by_user: long (nullable = true)
 |    |    |-- comment: string (nullable = true)
 |    |    |-- date: string (nullable = true)
 |    |    |-- discnumber: string (nullable = true)
 |    |    |-- duration: long (nullable = true)
 |    |    |-- duration_ms: long (nullable = true)
 |    |    |-- dur

In [5]:

# 1. Chuyển timestamp sang ngày tháng
# 2. Xử lý các cột lồng nhau (Nested Columns)
# Giả sử cấu trúc như mẫu bạn gửi: track_metadata.artist_name

df_clean = df.withColumn("event_date", from_unixtime(col("timestamp")).cast("date")) \
             .withColumn("artist_name", col("track_metadata.artist_name")) \
             .withColumn("track_name", col("track_metadata.track_name")) \
             .select("user_id", "event_date", "artist_name", "track_name")

# Xem thử 5 dòng đầu
df_clean.show(5)

+-------+----------+-----------------+--------------------+
|user_id|event_date|      artist_name|          track_name|
+-------+----------+-----------------+--------------------+
|  90803|2009-04-12|Tribe of Issachar|        Tribal Natty|
|  90803|2009-04-12|      Snake Thing|              scorch|
|  90803|2009-04-12|          Rainbow|     L.A. Connection|
|  90803|2009-04-12|   Built to Spill|Fly Around My Pre...|
|  90803|2009-04-12|       Herbaliser|Wall Crawling Gia...|
+-------+----------+-----------------+--------------------+
only showing top 5 rows

